In [ ]:
import os
import logging

import requests
import yfinance as yf
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup


logging.getLogger('yfinance').setLevel(logging.CRITICAL)

url = 'https://q.stock.sohu.com/cn/bk_4539.html'

response = requests.get(url)

# 使用BeautifulSoup解析HTML
soup = BeautifulSoup(response.text, 'html.parser')

# 初始化股票字典
stock_dict = {}

# 遍历所有包含股票代码和名称的<tr>元素
for tr in soup.find_all('tr', class_='red'):
    # 提取股票代码
    code = tr.find('td', class_='e1').text
    
    # 判断股票在哪个交易所，给股票代码加上后缀
    if code[0] == '6':
        code = code + '.SS'
    else:
        code = code + '.SZ'

    # 提取股票名称，注意股票名称在<a>标签内
    name = tr.find('td', class_='e2').a.text
    # 将股票代码和名称存入字典
    stock_dict[code] = name

print(stock_dict)

stock_code = list(stock_dict.keys())
stock_code.sort()

print(len(stock_code))

In [ ]:
csv_file = "stocks3.csv"

# 删除原有的csv文件
try:
    os.remove(csv_file)
except:
    pass

# 初始化一个空的DataFrame列表
data_frames = []
# 初始化一个空的错误代码列表
error_codes = []

for code in tqdm(stock_code, desc="Downloading stock data"):
    # 爬取近一年的股票数据
    data = yf.download(code, start="2023-01-01", end="2024-01-01", progress=False)
    if data.empty:
        error_codes.append(code)
        continue
    # 仅保留收盘价数据
    close_data = data[["Close"]]
    # 重命名列名为股票名称
    close_data.columns = [stock_dict[code]]
    # 将索引转换为字符串格式
    close_data.index = close_data.index.strftime('%Y/%m/%d')
    # 将DataFrame添加到列表中
    data_frames.append(close_data)


print(f'找到{len(data_frames)-len(error_codes)}个股票的数据')

# 如果某个股票存在一个日期为空值，就删除这个股票
data_frames.dropna(axis=1, how='any', inplace=True)

# 重新计算股票数量
print(f'有效股票数量：{len(data_frames)}')

# 使用pd.concat一次性合并所有DataFrame
all_data = pd.concat(data_frames, axis=1)

# 将最终的DataFrame写入CSV文件
all_data.to_csv(csv_file, encoding='utf-8-sig')